In [1]:
import pickle
import numpy as np
import pandas as pd
# 指定.pkl文件的路径
# file_path = 'data/census13/original.pkl'
# file_path = 'data/census13/workload/base.pkl'
# file_path = 'data/census13/workload/base-original-label.pkl'
file_path = 'data/census13/original.table.pkl'
# file_path = 'data/census13/lw/original_base_200_123.pkl'
# file_path = 'data/census13/workload/lstm-original-label.pkl'
# file_path = 'data/census13/workload/lstm-small-original-label.pkl'
# file_path = 'data/census13/workload/lstm-small.pkl'



# 打开文件并读取数据
with open(file_path, 'rb') as file:
    table = pickle.load(file)

In [2]:
# 当前这些列的组合没有生成过1w维
# 按照列的顺序，排列所有行
col_list = ["workclass", "capital_loss"]
selected_data = table.data[col_list].reset_index(drop=True)
df_sorted = selected_data.sort_values(by=col_list).reset_index(drop=True)
# 使用 value_counts 统计每个组合的数量
grouped_counts = df_sorted.groupby(col_list).size().reset_index(name='count')
grouped_counts = grouped_counts[grouped_counts['count'] > 0]
# 计算累计概率cumulative_probability
# grouped_counts包含[col_list每列对应的值, count, cumulative_probability]
grouped_counts['cumulative_probability'] = grouped_counts['count'].cumsum() / len(df_sorted)


In [3]:
grouped_counts


,workclass,capital_loss,count,cumulative_probability
0,?,0,2698,0.055239
4,?,419,2,0.055280
12,?,1258,1,0.055301
15,?,1408,1,0.055321
16,?,1411,1,0.055342
...,...,...,...,...
779,State-gov,2472,1,0.999529
785,State-gov,2824,1,0.999550
788,State-gov,3683,1,0.999570
792,Without-pay,0,20,0.999980


In [4]:
table.columns["age"].dtype

dtype('int64')

In [5]:
import pandas as pd
from itertools import product
from lecarb.dtypes import is_categorical

# 获取col_list中的所有不同值
# unique_values = [pd.unique(grouped_counts[col].values) for col in col_list]

# 根据数据类型获取unique_values
unique_values = []
for col in col_list:
    if is_categorical(table.columns[col].dtype):
        unique_values.append(pd.unique(grouped_counts[col].values))
    else:
        unique_values.append(range(table.columns[col].minval, table.columns[col].maxval + 1))


In [6]:
unique_values

[[?, Federal-gov, Local-gov, Never-worked, Private, Self-emp-inc, Self-emp-not-inc, State-gov, Without-pay]
 Categories (9, object): [? < Federal-gov < Local-gov < Never-worked ... Self-emp-inc < Self-emp-not-inc < State-gov < Without-pay],
 range(0, 4357)]

In [7]:
# 生成所有可能的组合
cartesian_product = list(product(*unique_values))

cartesian_product

[('?', 0),
 ('?', 1),
 ('?', 2),
 ('?', 3),
 ('?', 4),
 ('?', 5),
 ('?', 6),
 ('?', 7),
 ('?', 8),
 ('?', 9),
 ('?', 10),
 ('?', 11),
 ('?', 12),
 ('?', 13),
 ('?', 14),
 ('?', 15),
 ('?', 16),
 ('?', 17),
 ('?', 18),
 ('?', 19),
 ('?', 20),
 ('?', 21),
 ('?', 22),
 ('?', 23),
 ('?', 24),
 ('?', 25),
 ('?', 26),
 ('?', 27),
 ('?', 28),
 ('?', 29),
 ('?', 30),
 ('?', 31),
 ('?', 32),
 ('?', 33),
 ('?', 34),
 ('?', 35),
 ('?', 36),
 ('?', 37),
 ('?', 38),
 ('?', 39),
 ('?', 40),
 ('?', 41),
 ('?', 42),
 ('?', 43),
 ('?', 44),
 ('?', 45),
 ('?', 46),
 ('?', 47),
 ('?', 48),
 ('?', 49),
 ('?', 50),
 ('?', 51),
 ('?', 52),
 ('?', 53),
 ('?', 54),
 ('?', 55),
 ('?', 56),
 ('?', 57),
 ('?', 58),
 ('?', 59),
 ('?', 60),
 ('?', 61),
 ('?', 62),
 ('?', 63),
 ('?', 64),
 ('?', 65),
 ('?', 66),
 ('?', 67),
 ('?', 68),
 ('?', 69),
 ('?', 70),
 ('?', 71),
 ('?', 72),
 ('?', 73),
 ('?', 74),
 ('?', 75),
 ('?', 76),
 ('?', 77),
 ('?', 78),
 ('?', 79),
 ('?', 80),
 ('?', 81),
 ('?', 82),
 ('?', 83),
 (

In [8]:
# 创建新的DataFrame
result_df = pd.DataFrame(cartesian_product, columns=col_list)
result_df

,workclass,capital_loss
0,?,0
1,?,1
2,?,2
3,?,3
4,?,4
...,...,...
39208,Without-pay,4352
39209,Without-pay,4353
39210,Without-pay,4354
39211,Without-pay,4355


In [9]:

# 增加一列index_alldomain，内容为新的索引值
result_df['index_alldomain'] = pd.Series(range(len(result_df))) / len(result_df)
# 乘以10000后四舍五入取整
result_df['index_alldomain'] = round(result_df['index_alldomain'] * 10000)
# 排序新的DataFrame
result_df.sort_values(by=col_list, inplace=True)
result_df.reset_index(drop=True, inplace=True)
result_df

,workclass,capital_loss,index_alldomain
0,?,0,0.0
1,?,1,0.0
2,?,2,1.0
3,?,3,1.0
4,?,4,1.0
...,...,...,...
39208,Without-pay,4352,9999.0
39209,Without-pay,4353,9999.0
39210,Without-pay,4354,9999.0
39211,Without-pay,4355,9999.0


In [10]:
# 合并原始的grouped_counts中的count和cumulative_probability列，只合并存在的行
# 合并原始的grouped_counts中的count和cumulative_probability列，只保留存在的行
grouped_counts = pd.merge(result_df, grouped_counts, how='inner', on=col_list)
grouped_counts.columns

Index(['workclass', 'capital_loss', 'index_alldomain', 'count',
       'cumulative_probability'],
      dtype='object')

In [11]:
grouped_counts

,workclass,capital_loss,index_alldomain,count,cumulative_probability
0,?,0,0.0,2698,0.055239
1,?,419,107.0,2,0.055280
2,?,1258,321.0,1,0.055301
3,?,1408,359.0,1,0.055321
4,?,1411,360.0,1,0.055342
...,...,...,...,...,...
345,State-gov,2472,8408.0,1,0.999529
346,State-gov,2824,8498.0,1,0.999550
347,State-gov,3683,8717.0,1,0.999570
348,Without-pay,0,8889.0,20,0.999980


In [2]:
import numpy as np

# 假设你有一个50000*12的数组，命名为data
data = np.random.rand(10, 3)
print(data)
# 将数组重塑为1000*50*12的结构
reshaped_data = data.reshape(5, 2, 3)

# 输出重塑后的数组形状
print(reshaped_data.shape)
print(reshaped_data)


[[0.42131775 0.35425925 0.94795996]
 [0.85651308 0.23342353 0.28667997]
 [0.61662599 0.30646846 0.43598051]
 [0.50160331 0.79034753 0.93887179]
 [0.80360227 0.13460084 0.11978511]
 [0.59347961 0.22193423 0.28477896]
 [0.50336218 0.43128274 0.11596517]
 [0.25901304 0.24787388 0.46458154]
 [0.6189575  0.30014948 0.78323467]
 [0.7006315  0.18008172 0.41695876]]
(5, 2, 3)
[[[0.42131775 0.35425925 0.94795996]
  [0.85651308 0.23342353 0.28667997]]

 [[0.61662599 0.30646846 0.43598051]
  [0.50160331 0.79034753 0.93887179]]

 [[0.80360227 0.13460084 0.11978511]
  [0.59347961 0.22193423 0.28477896]]

 [[0.50336218 0.43128274 0.11596517]
  [0.25901304 0.24787388 0.46458154]]

 [[0.6189575  0.30014948 0.78323467]
  [0.7006315  0.18008172 0.41695876]]]
